# Count the signs

* ~~Write out the video output file. - this may need to be done inside the yolo file.~~
* Analyse the file, to get some idea of approach to counting.
* Extend the model so that it has an extra dictionary for the entry for the output so that it can overwrite the answer it provides.
* Then write out the end file so that 

In [1]:
#read the file in so that it pushes each found object into a dictionary with the frame as the key.
import os
base_dir = '/Users/datascience4/documents/yolo3/keras-yolo3/output/'

file = 'B4087_NB_RAV_R07_180519074106_completed_counting.txt'

file = open(os.path.join(base_dir, file), 'r')
file_read = file.readlines()
file.close()

In [2]:
# print(file_read)

In [47]:
import re
dic = {}
#leave the frame outside.
frame = {}
for line in file_read:
    ## for frame, shape read
    if 'Frame' in line:
        new_line = list(line[:-1])
        frame_name = ''.join(str(letter) for letter in new_line)
        frame[frame_name] = {}
        frame[frame_name]['frame'] = int(frame_name[6:])
        if frame_name in line:
            line_count = file_read.index(line)
    if line == '(416, 416, 3)\n':
        line = list(line[:-1])
        frame[frame_name]['shape'] = ''.join(str(letter) for letter in line)
    if 'Found' in line:
        line = list(line)
        number_end = line.index('b') - 1
        number = line[6:number_end]
        frame[frame_name]['objects_count'] = int(''.join(str(n) for n in number))
        if frame[frame_name]['objects_count'] > 0:
            frame[frame_name]['objects'] = {}
            for i in range(frame[frame_name]['objects_count']):
                obj_line = (line_count + 2) + (i + 1)
                object_dic = frame[frame_name]['objects']['object ' + str(i + 1)] = {}
                object_list = list(file_read[obj_line])

                obj_list = file_read[obj_line]
                sign_name = re.search('(\w+\s)+', obj_list)
                if sign_name != None:
                    sign_name = sign_name.group()[:-1]
                    confidence_score = obj_list[(len(sign_name)+1):(len(sign_name) + 5)]
                    coord_pattern = r'\(([0-9])+\,\s([0-9])+\)'
                    left_top_coord = re.search(r'\(([0-9])+\,\s([0-9])+\)', obj_list).group()
                    left_top_x = re.search(r'([0-9])+', left_top_coord).group()
                    left_top_y = re.search(r'(?<=\,\s)([0-9])+', left_top_coord).group()
                    right_bottom_coord = re.search(
                        r'(?<=\)\s)\(([0-9])+\,\s([0-9])+\)', obj_list).group()
                    right_bottom_x = re.search(r'([0-9])+', right_bottom_coord).group()
                    right_bottom_y = re.search(r'(?<=\,\s)([0-9])+', right_bottom_coord).group()                        
                ## push it all to the object dictionary
                object_dic['class'] = sign_name
                object_dic['confidence_score'] = float(confidence_score)
                object_dic['left_top_coord'] = left_top_coord
                object_dic['right_bottom_coord'] = right_bottom_coord
                object_dic['left_top_x'] = int(left_top_x)
                object_dic['left_top_y'] = int(left_top_y)
                object_dic['right_bottom_x'] = int(right_bottom_x)
                object_dic['right_bottom_y'] = int(right_bottom_y)
            obj_line = (line_count + 2) + (frame[frame_name]['objects_count'] + 1)
            detection_time = float(file_read[obj_line])
#             print(obj_line, detection_time)
            frame[frame_name]['detection_time'] = detection_time
        else:
            ## add time to collection
            obj_line = (line_count + 3)
            detection_time = float(file_read[obj_line])
#             print('no detection', obj_line, detection_time)
            frame[frame_name]['detection_time'] = detection_time


In [48]:
frame['Frame 2']

{'frame': 2,
 'shape': '(416, 416, 3)',
 'objects_count': 0,
 'detection_time': 0.9735546739993879}

In [49]:
frame['Frame 1688']

{'frame': 1688,
 'shape': '(416, 416, 3)',
 'objects_count': 4,
 'objects': {'object 1': {'class': 'keep left sign',
   'confidence_score': 0.66,
   'left_top_coord': '(1061, 302)',
   'right_bottom_coord': '(1108, 395)',
   'left_top_x': 1061,
   'left_top_y': 302,
   'right_bottom_x': 1108,
   'right_bottom_y': 395},
  'object 2': {'class': 'speed sign',
   'confidence_score': 0.43,
   'left_top_coord': '(962, 0)',
   'right_bottom_coord': '(1141, 106)',
   'left_top_x': 962,
   'left_top_y': 0,
   'right_bottom_x': 1141,
   'right_bottom_y': 106},
  'object 3': {'class': 'speed sign',
   'confidence_score': 0.58,
   'left_top_coord': '(503, 126)',
   'right_bottom_coord': '(550, 168)',
   'left_top_x': 503,
   'left_top_y': 126,
   'right_bottom_x': 550,
   'right_bottom_y': 168},
  'object 4': {'class': 'speed sign',
   'confidence_score': 0.78,
   'left_top_coord': '(1089, 141)',
   'right_bottom_coord': '(1128, 191)',
   'left_top_x': 1089,
   'left_top_y': 141,
   'right_bottom_

In [43]:
# print(line_count)

In [44]:
# for k, v in frame.items():
#     print(k, v)

In [45]:
# type(frame['Frame 2596']['objects']['object 1']['confidence_score'])

In [46]:
## now to address the chain of events for counting
## think about the flow of data
## read the flow of counts into a list
## as the list enters, we only want to keep the chain of 10 frames
frames_list_found = []
found_plus_next_ten = []
for k, v in frame.items():
    i = 0
    if v['objects_count'] > 0:
        for j in range(v['objects_count']):
            i += 1
            obj = v['objects']['object ' + str(i)]
            frames_list_found += [frame[k]['frame'], obj['confidence_score']]
            ten_frames = []
            for l in range(10):
                obj_mod = frame['Frame ' + str(l + frame[k]['frame'])]
#                 print(obj_mod)
                ten_frames.append(obj_mod)
#                 print('this', obj_mod)
#                 if obj_mod['objects_count'] > 0:
#                     for m in range(obj_mod['objects_count']):
#                         obj_mod = obj_mod['objects']['object ' + str(i)]
#                         print('obj_mod?', obj_mod)
#                         ten_frames.append([[obj_mod['frame'], obj_mod['confidence_score']]])
#                         print(l + obj_mod['frame'])
#             print('end\n')
            found_plus_next_ten.append(ten_frames)
#             print(found_plus_next_ten)

In [50]:
# print(len(found_plus_next_ten))
count = 0
for k, v in frame.items():
    if frame[k]['objects_count'] > 0:
        count += frame[k]['objects_count']
# print(count)
# for i in range(len(found_plus_next_ten[0])):
#     print(found_plus_next_ten[0][i])

# Do we want to count the difference in frequency between objects detected, via summary statistics?
Yes (frequency of 1 object being detected, frequency of multiple objects detected)

In [51]:
def intersection_over_union(boxA, boxB):
	"""left, top, right bottom"""
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	#compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	#compute the area of both the prediction and ground truth
	boxA_area = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxB_area = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	#compute the intersection over union by taking the intersection
	#area and dividing it by the sum of prediction + ground-truth
	#areas - the intersection area
	iou = interArea / float(boxA_area + boxB_area - interArea)
	# return the intersection over union value
	return iou

def larger_intersection_over_union(boxA, boxB):
	"""
    left, top, right bottom, amending so that it becomes an imaginary box in nature
    possibly required for right side, or turning where the extremes to the left or right should be larger
    IOU?
    
    It might be incorrect.
    """
	amended_boxA = list(boxA)
	amended_boxA[0] = amended_boxA[0] - 200
	amended_boxA[2] = amended_boxA[2] + 200
	xA = max(amended_boxA[0], boxB[0])
	yA = max(amended_boxA[1], boxB[1])
	xB = min(amended_boxA[2], boxB[2])
	yB = min(amended_boxA[3], boxB[3])
	#compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	#compute the area of both the prediction and ground truth
	boxA_area = (
		amended_boxA[2] - amended_boxA[0] + 1) * (amended_boxA[3] - amended_boxA[1] + 1)
	boxB_area = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	#compute the intersection over union by taking the intersection
	#area and dividing it by the sum of prediction + ground-truth
	#areas - the intersection area
	iou = interArea / float(boxA_area + boxB_area - interArea)
	# return the intersection over union value
	return iou

In [52]:
## do they need to appear in sequence?
## assume that they do for now
## also how many objects in each item?
## need to ensure that what is passed in


## Notes for LL
## Approach is to identify where an object is identified, push that frame and the next 9 into a list
## then go through the list, and commpare each frame to the next where the object counts follow one another
## then for each object compare each object with each other object until all detected objects have an IOU calculated
## if the IOU is not zero, and the classes match, print the output.


## Is the general approach correct?

## steps that need to be done.
## take into account the confidence score? of the item itself
## where it's true that each frame supercedes a previous frame to drop the first frame
## only when it's true that the frames were in order.

##total count IOU is too simple
total_count_IOU = 0
intersection_boxes_criteria_met = []
# not used yet
intersection_boxes_not_met = []
intersection_boxes_met_not_confidence = []
for i in range(len(found_plus_next_ten)):
    for j in range(len(found_plus_next_ten[i]) - 1):
        boxA = found_plus_next_ten[i][j]
        boxB = found_plus_next_ten[i][j + 1]
        if boxA['objects_count'] > 0 and boxB['objects_count'] > 0:
            for k in range(boxA['objects_count']):
                for l in range(boxB['objects_count']):
                    new_boxA = [
                        boxA['objects']['object ' + str(k+1)]['left_top_x'],
                        boxA['objects']['object ' + str(k+1)]['left_top_y'],
                        boxA['objects']['object ' + str(k+1)]['right_bottom_x'],
                        boxA['objects']['object ' + str(k+1)]['right_bottom_y'],
                    ]
                    box_a_class = boxA['objects']['object ' + str(k+1)]['class']
                    box_a_confidence = boxA['objects']['object ' + str(k+1)]['confidence_score']
                    new_boxB = [
                        boxB['objects']['object ' + str(l + 1)]['left_top_x'],
                        boxB['objects']['object ' + str(l + 1)]['left_top_y'],
                        boxB['objects']['object ' + str(l + 1)]['right_bottom_x'],
                        boxB['objects']['object ' + str(l + 1)]['right_bottom_y']
                    ]
                    box_b_class = boxB['objects']['object ' + str(l + 1)]['class']
                    box_b_confidence = boxB['objects']['object ' + str(l+1)]['confidence_score']
                    if(intersection_over_union(new_boxA, new_boxB) != 0.0) \
                            and box_a_class == box_b_class \
                            and box_a_confidence > 0.60 \
                            and box_b_confidence > 0.60:
#                         print(new_boxA, '\n', new_boxB)
#                         print(intersection_over_union(new_boxA, new_boxB))
                        total_count_IOU += 1
                        intersection_boxes_criteria_met += [intersection_over_union(new_boxA, new_boxB)]
                    else:
                        if intersection_over_union(new_boxA, new_boxB) != 0.0:
                            intersection_boxes_met_not_confidence += [intersection_over_union(new_boxA, new_boxB)]
                        else:
                            intersection_boxes_not_met += [intersection_over_union(new_boxA, new_boxB)]
                        ## super lazy way of doing this
#                         boxA = found_plus_next_ten[i][k]
#                         boxB = found_plus_next_ten[i][l]
#                         #this bit is now incorrect.
#                         print('boxA object', k, boxA['frame'], 'boxB object', l, boxB['frame'])
#                         print('boxA confidence', boxA['objects']['object 1']['confidence_score'], '\n')
#                         print('boxB confidence', boxB['objects']['object 1']['confidence_score'], '\n')

## it's currently counting the same IOU's, so if I include the frames then drop them that should work.
print('IOU count occurences', total_count_IOU)
print('IOU conditions met list, confidence > 0.6', len(intersection_boxes_criteria_met))
# print('IOU conditions not met', len(intersection_boxes_criteria_not_met))
print('IOU condition met, but confidence not', len(intersection_boxes_met_not_confidence))
print('IOU condition not met', len(intersection_boxes_not_met))

IOU count occurences 55
IOU conditions met list, confidence > 0.6 55
IOU condition met, but confidence not 254
IOU condition not met 1843


In [53]:
# print(found_plus_next_ten[233][0])
# print(found_plus_next_ten[233][1])
a = found_plus_next_ten[233][0]
b = found_plus_next_ten[233][1]
for i in range(a['objects_count']):
    for j in range(b['objects_count']):
        boxA = a['objects']['object ' + str(i+1)]['left_top_x']
        boxB = b['objects']['object ' + str(j+1)]['left_top_x']
#         print(boxA, boxB, i+1 , j+1)

In [54]:
intersection_over_union([153, 79, 225, 172], [54,43,165,155])

0.0540584327914889

## Centroid option is likely a better solution.

In [55]:
#install the correct packages
from scipy.spatial import distance as dist
from collections import OrderedDict
import imutils
import numpy as np

In [66]:
# ?dist.euclidean
dist.euclidean([70,23],[123, 130])
dist.cdist

119.40686747419514

In [65]:
objects = OrderedDict()
objectID = 0

OrderedDict(frame)

OrderedDict([('Frame 1',
              {'frame': 1,
               'shape': '(416, 416, 3)',
               'objects_count': 0,
               'detection_time': 1.9706918119991315}),
             ('Frame 2',
              {'frame': 2,
               'shape': '(416, 416, 3)',
               'objects_count': 0,
               'detection_time': 0.9735546739993879}),
             ('Frame 3',
              {'frame': 3,
               'shape': '(416, 416, 3)',
               'objects_count': 0,
               'detection_time': 0.9658381970002665}),
             ('Frame 4',
              {'frame': 4,
               'shape': '(416, 416, 3)',
               'objects_count': 0,
               'detection_time': 1.0914586590006365}),
             ('Frame 5',
              {'frame': 5,
               'shape': '(416, 416, 3)',
               'objects_count': 0,
               'detection_time': 0.981479070000205}),
             ('Frame 6',
              {'frame': 6,
               'shape': '(416, 416